In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import warnings

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import seaborn as sns
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
def missingness_summary(df, print_log=False, sort="none"):
    summary = df.apply(lambda x: x.isna().sum() / x.shape[0])

    if print_log == True:
        if sort == "none":
            print(summary)
        elif sort == "ascending":
            print(summary.sort_values())
        elif sort == "descending":
            print(summary.sort_values(ascending=False))
        else:
            print("Invalid value for sort parameter.")

    return

<IPython.core.display.Javascript object>

In [4]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [5]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

<IPython.core.display.Javascript object>

In [6]:
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
houseprices_df = pd.read_sql_query("select * from houseprices", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

# Data Cleaning
* Drop columns with majority missing information
    * `poolqc`, `miscfeature`, `alley`, `fence`
* Fill missing values in categorical variable with 'None`

In [7]:
houseprices_df.info()
houseprices_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

In [8]:
missingness_summary(houseprices_df, print_log=True, sort="descending")

poolqc           0.995205
miscfeature      0.963014
alley            0.937671
fence            0.807534
fireplacequ      0.472603
                   ...   
centralair       0.000000
salecondition    0.000000
heating          0.000000
totalbsmtsf      0.000000
id               0.000000
Length: 81, dtype: float64


<IPython.core.display.Javascript object>

In [9]:
houseprices_df = houseprices_df.drop(
    columns=["poolqc", "miscfeature", "alley", "fence"], axis=1
)

<IPython.core.display.Javascript object>

In [10]:
missingness_summary(
    houseprices_df.select_dtypes(include=["object"]), print_log=True, sort="descending"
)

missing_cat_cols = [
    "fireplacequ",
    "garagecond",
    "garagequal",
    "garagefinish",
    "garagetype",
    "bsmtexposure",
    "bsmtfintype2",
    "bsmtqual",
    "bsmtfintype1",
    "bsmtcond",
    "masvnrtype",
    # "electrical",  # most common not none
]

fireplacequ      0.472603
garagecond       0.055479
garagequal       0.055479
garagefinish     0.055479
garagetype       0.055479
bsmtexposure     0.026027
bsmtfintype2     0.026027
bsmtqual         0.025342
bsmtfintype1     0.025342
bsmtcond         0.025342
masvnrtype       0.005479
electrical       0.000685
condition1       0.000000
neighborhood     0.000000
condition2       0.000000
bldgtype         0.000000
salecondition    0.000000
lotconfig        0.000000
landslope        0.000000
roofstyle        0.000000
utilities        0.000000
landcontour      0.000000
lotshape         0.000000
street           0.000000
housestyle       0.000000
foundation       0.000000
roofmatl         0.000000
exterior1st      0.000000
exterior2nd      0.000000
exterqual        0.000000
extercond        0.000000
saletype         0.000000
heating          0.000000
heatingqc        0.000000
centralair       0.000000
kitchenqual      0.000000
functional       0.000000
paveddrive       0.000000
mszoning    

<IPython.core.display.Javascript object>

In [11]:
for col in missing_cat_cols:
    print(houseprices_df[col].value_counts(dropna=False))

NaN    690
Gd     380
TA     313
Fa      33
Ex      24
Po      20
Name: fireplacequ, dtype: int64
TA     1326
NaN      81
Fa       35
Gd        9
Po        7
Ex        2
Name: garagecond, dtype: int64
TA     1311
NaN      81
Fa       48
Gd       14
Ex        3
Po        3
Name: garagequal, dtype: int64
Unf    605
RFn    422
Fin    352
NaN     81
Name: garagefinish, dtype: int64
Attchd     870
Detchd     387
BuiltIn     88
NaN         81
Basment     19
CarPort      9
2Types       6
Name: garagetype, dtype: int64
No     953
Av     221
Gd     134
Mn     114
NaN     38
Name: bsmtexposure, dtype: int64
Unf    1256
Rec      54
LwQ      46
NaN      38
BLQ      33
ALQ      19
GLQ      14
Name: bsmtfintype2, dtype: int64
TA     649
Gd     618
Ex     121
NaN     37
Fa      35
Name: bsmtqual, dtype: int64
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
NaN     37
Name: bsmtfintype1, dtype: int64
TA     1311
Gd       65
Fa       45
NaN      37
Po        2
Name: bsmtcond, dtype: i

<IPython.core.display.Javascript object>

In [12]:
houseprices_df[missing_cat_cols] = houseprices_df[missing_cat_cols].fillna(value="None")
houseprices_df["electrical"] = houseprices_df["electrical"].fillna(value="SBrkr")

<IPython.core.display.Javascript object>

In [13]:
missingness_summary(
    houseprices_df.select_dtypes(exclude=["object"]), print_log=True, sort="descending"
)

lotfrontage      0.177397
garageyrblt      0.055479
masvnrarea       0.005479
bsmtfinsf1       0.000000
lowqualfinsf     0.000000
secondflrsf      0.000000
firstflrsf       0.000000
totalbsmtsf      0.000000
bsmtunfsf        0.000000
bsmtfinsf2       0.000000
saleprice        0.000000
bsmtfullbath     0.000000
yearremodadd     0.000000
yearbuilt        0.000000
overallcond      0.000000
overallqual      0.000000
lotarea          0.000000
mssubclass       0.000000
grlivarea        0.000000
bsmthalfbath     0.000000
yrsold           0.000000
fullbath         0.000000
mosold           0.000000
miscval          0.000000
poolarea         0.000000
screenporch      0.000000
threessnporch    0.000000
enclosedporch    0.000000
openporchsf      0.000000
wooddecksf       0.000000
garagearea       0.000000
garagecars       0.000000
fireplaces       0.000000
totrmsabvgrd     0.000000
kitchenabvgr     0.000000
bedroomabvgr     0.000000
halfbath         0.000000
id               0.000000
dtype: float

<IPython.core.display.Javascript object>

In [14]:
# borrowed from kaggle, fills with median values
houseprices_df['garageyrblt'] = houseprices_df['garageyrblt'].fillna(houseprices_df['yrsold']-35)
houseprices_df['lotfrontage'] = houseprices_df['lotfrontage'].fillna(68)


<IPython.core.display.Javascript object>

In [15]:
missingness_summary(houseprices_df, print_log=True, sort="descending")

masvnrarea      0.005479
saleprice       0.000000
extercond       0.000000
roofstyle       0.000000
roofmatl        0.000000
                  ...   
kitchenabvgr    0.000000
kitchenqual     0.000000
totrmsabvgrd    0.000000
functional      0.000000
id              0.000000
Length: 77, dtype: float64


<IPython.core.display.Javascript object>

In [16]:
houseprices_df["masvnrtype"] = houseprices_df["masvnrtype"].fillna(value=0)
houseprices_df["masvnrarea"] = houseprices_df["masvnrarea"].fillna(value=0)

<IPython.core.display.Javascript object>

# Feature Engineering and Selection

In [23]:
houseprices_df["houseage"] = houseprices_df["yrsold"] - houseprices_df["yearbuilt"]
houseprices_df["total_baths"] = (
    houseprices_df["fullbath"]
    + 0.5 * houseprices_df["halfbath"]
    + houseprices_df["bsmtfullbath"]
    + 0.5 * houseprices_df["bsmthalfbath"]
)

houseprices_df["totalarea"] = (
    houseprices_df["totalbsmtsf"]
    + houseprices_df["firstflrsf"]
    + houseprices_df["secondflrsf"]
)



<IPython.core.display.Javascript object>

In [97]:
houseprices_df.corr()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,...,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,houseage,total_baths,totalarea
id,1.000000,0.011156,-0.010224,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.051071,-0.005024,...,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917,0.012724,0.004540,-0.000322
mssubclass,0.011156,1.000000,-0.356286,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.023573,-0.069836,...,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284,-0.028746,0.151048,-0.082225
lotfrontage,-0.010224,-0.356286,1.000000,0.302265,0.235326,-0.053703,0.115781,0.083896,0.178194,0.212909,...,0.061555,0.037616,0.180716,-0.001610,0.010726,0.006009,0.334544,-0.115336,0.199280,0.426833
lotarea,-0.033226,-0.139781,0.302265,1.000000,0.105806,-0.005636,0.014228,0.013788,0.103321,0.214103,...,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843,-0.014832,0.204764,0.307234
overallqual,-0.028365,0.032628,0.235326,0.105806,1.000000,-0.091932,0.572323,0.550684,0.407252,0.239666,...,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982,-0.572629,0.541063,0.668155
overallcond,0.012609,-0.059316,-0.053703,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.125694,-0.046231,...,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856,0.377325,-0.174032,-0.143814
yearbuilt,-0.012713,0.027850,0.115781,0.014228,0.572323,-0.375983,1.000000,0.592855,0.311600,0.249503,...,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897,-0.999036,0.524298,0.347133
yearremodadd,-0.021998,0.040581,0.083896,0.013788,0.550684,0.073741,0.592855,1.000000,0.176529,0.128451,...,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101,-0.590360,0.443719,0.342873
masvnrarea,-0.051071,0.023573,0.178194,0.103321,0.407252,-0.125694,0.311600,0.176529,1.000000,0.261256,...,0.019144,0.062248,0.011928,-0.029512,-0.006723,-0.008317,0.472614,-0.311479,0.314248,0.444454
bsmtfinsf1,-0.005024,-0.069836,0.212909,0.214103,0.239666,-0.046231,0.249503,0.128451,0.261256,1.000000,...,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420,-0.248483,0.481609,0.415843


<IPython.core.display.Javascript object>

In [27]:
cols_of_interest = [
    "saleprice",
    "mszoning",
    "bldgtype",
    "housestyle",
    "overallcond",
    "total_baths",  # some places have 0 bathrooms?
    "salecondition",
    "lotarea",
    "lotfrontage",
    "houseage",
    "garagecars",
    "totalarea",
    "poolarea",
]

# columns chosen somewhat (completely) arbitrarily

<IPython.core.display.Javascript object>

In [25]:
houses = houseprices_df[cols_of_interest]
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   saleprice      1460 non-null   int64  
 1   mszoning       1460 non-null   object 
 2   bldgtype       1460 non-null   object 
 3   housestyle     1460 non-null   object 
 4   overallcond    1460 non-null   int64  
 5   total_baths    1460 non-null   float64
 6   salecondition  1460 non-null   object 
 7   lotarea        1460 non-null   int64  
 8   lotfrontage    1460 non-null   float64
 9   houseage       1460 non-null   int64  
 10  garagecars     1460 non-null   int64  
 11  totalarea      1460 non-null   int64  
 12  poolarea       1460 non-null   int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 148.4+ KB


<IPython.core.display.Javascript object>

Seperate target from features, then split

In [86]:
X = houses.drop(columns='saleprice')
y = houses['saleprice']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


<IPython.core.display.Javascript object>

Transform and Treat columns

In [87]:
cat_cols = X.select_dtypes(include=["object"]).columns.to_list()
drop_cats = []

num_cols = X.select_dtypes(exclude=["object"]).columns.to_list()
cat_cols
num_cols

['overallcond',
 'total_baths',
 'lotarea',
 'lotfrontage',
 'houseage',
 'garagecars',
 'totalarea',
 'poolarea']

<IPython.core.display.Javascript object>

In [88]:
drop_cats = []
for col in cat_cols:
    drop_cats.append(houses[col].mode().values[0])
drop_cats

['RL', '1Fam', '1Story', 'Normal']

<IPython.core.display.Javascript object>

In [89]:
ct = ColumnTransformer(
    [("one_hot_encode", OneHotEncoder(drop=drop_cats, sparse=False), cat_cols)],
    remainder="passthrough",
)

<IPython.core.display.Javascript object>

In [90]:
ct.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('one_hot_encode',
                                 OneHotEncoder(categories='auto',
                                               drop=['RL', '1Fam', '1Story',
                                                     'Normal'],
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 ['mszoning', 'bldgtype', 'housestyle',
                                  'salecondition'])],
                  verbose=False)

<IPython.core.display.Javascript object>

In [91]:
X_train_trans = ct.transform(X_train)
X_test_trans = ct.transform(X_test)

<IPython.core.display.Javascript object>

In [92]:
X_train_trans.shape

(1168, 28)

<IPython.core.display.Javascript object>

In [93]:
cat_names = ct.transformers_[0][1].get_feature_names(cat_cols)
cat_names = list(cat_names)
new_col_names = cat_names + num_cols

X_train = pd.DataFrame(X_train_trans, columns=new_col_names)
X_test = pd.DataFrame(X_test_trans, columns=new_col_names)

<IPython.core.display.Javascript object>

In [95]:
print_vif(X_train)

VIF results
-------------------------------
const                    65.903529
mszoning_C (all)          1.067229
mszoning_FV               1.177840
mszoning_RH               1.080115
mszoning_RM               1.546291
bldgtype_2fmCon           1.100172
bldgtype_Duplex           1.232440
bldgtype_Twnhs            1.292049
bldgtype_TwnhsE           1.382988
housestyle_1.5Fin         1.397993
housestyle_1.5Unf         1.052035
housestyle_2.5Fin         1.100095
housestyle_2.5Unf         1.137121
housestyle_2Story         1.419540
housestyle_SFoyer         1.108348
housestyle_SLvl           1.096461
salecondition_Abnorml     1.067821
salecondition_AdjLand     1.062481
salecondition_Alloca      1.152710
salecondition_Family      1.028764
salecondition_Partial     1.231499
overallcond               1.261950
total_baths               2.275485
lotarea                   1.206124
lotfrontage               1.612836
houseage                  2.988885
garagecars                1.830813
totalarea  

<IPython.core.display.Javascript object>

No features are highly colinear, so model building can begin

# Modeling House Prices


In [96]:
linear = LinearRegression()
linear.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

<IPython.core.display.Javascript object>

In [ ]:
grid = {"alpha": [0.0001, 0.001, 0.01, 0.1]}
ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)